# NB01: シフトパラメータの必要性 — HCurl curl-curl 問題

## 目的

HCurl有限要素法による curl-curl 問題では、剛性行列が**半正定値**（ゼロ固有値を持つ）になる。
このため、不完全コレスキー (IC) 分解が数値的に破綻する可能性がある。

本ノートブックでは:
1. **シフトパラメータ** (`shift`) と **auto-shift** がIC分解を安定化するメカニズムを解説
2. Unit Cube上のcurl-curl問題で、shifted-ICCGの動作を検証
3. BDDCとの性能比較

## 理論: IC分解とシフトパラメータ

### Curl-Curl 行列の性質

HCurl空間で離散化した curl-curl 行列:

$$A_{ij} = \int_\Omega 
abla 	imes N_i \cdot 
abla 	imes N_j \, dx + arepsilon \int_\Omega N_i \cdot N_j \, dx$$

- $arepsilon = 0$ のとき、$A$ は**半正定値**: $\ker(A) = \{
abla \phi\}$（勾配場）
- `nograds=True` で勾配自由度を除外しても、数値的にゼロに近い固有値が残存

### IC分解の破綻メカニズム

不完全コレスキー分解 $A pprox L D L^T$ では、対角要素を逐次計算:

$$d_i = a_{ii} - \sum_{k < i} l_{ik}^2 \, d_k^{-1}$$

$A$ が半正定値の場合、$d_i \leq 0$ となりうる → 分解が破綻する。

### シフトパラメータ $lpha$ による安定化

$d_i = lpha \cdot a_{ii} - \sum l_{ik}^2 d_k^{-1}$ として $lpha > 1$ を与えると、
対角優位性が強化されて $d_i > 0$ が保証される。

- `shift=1.05`: 標準的な値（5%の対角シフト）
- `auto_shift=True`: $|d_i|$ が閾値未満になったら $lpha$ を自動増加して再分解

### 微小正則化 $arepsilon$

$arepsilon > 0$ の質量項を加えると、行列は正定値になる。
$arepsilon$ が十分小さければ $	ext{curl}(A)$ の精度に影響しない。

In [1]:
import ngsolve
from ngsolve import *
from netgen.occ import *
import time

SetNumThreads(8)
from sparsesolv_ngsolve import SparseSolvSolver, BDDCPreconditioner
from ngsolve.krylovspace import CGSolver
print("Setup complete")

Setup complete


## 問題設定: Unit Cube curl-curl

$$
abla 	imes 
abla 	imes A + arepsilon A = J, \quad arepsilon = 10^{-6}$$

- 領域: $[-0.5, 0.5]^3$
- ソース: $J = (0, 0, 1)$
- 境界条件: Dirichlet (全面)
- HCurl order=2, `nograds=True`

In [2]:
# メッシュ作成
box = Box((-0.5,-0.5,-0.5), (0.5,0.5,0.5))
mesh = box.GenerateMesh(maxh=0.2).Curve(2)
print(f"メッシュ: {mesh.ne} 要素")

# 有限要素空間
eps = 1e-6
fes = HCurl(mesh, order=2, nograds=True, dirichlet=".*")
u, v = fes.TrialFunction(), fes.TestFunction()
a = BilinearForm(curl(u)*curl(v)*dx + eps*u*v*dx)
a.Assemble()
f = LinearForm(CoefficientFunction((0,0,1))*v*dx)
f.Assemble()
print(f"自由度数: {fes.ndof}")

# 直接法 (参照解)
t0 = time.time()
gfu_ref = GridFunction(fes)
gfu_ref.vec.data = a.mat.Inverse(fes.FreeDofs(), inverse="sparsecholesky") * f.vec
t_direct = time.time() - t0
B_ref = curl(gfu_ref)
B_norm = sqrt(Integrate(InnerProduct(B_ref, B_ref), mesh))
print(f"直接法: {t_direct:.3f}s, ||B|| = {B_norm:.6e}")

メッシュ: 646 要素
自由度数: 3976


直接法: 0.203s, ||B|| = 1.873738e-01


## 検証1: Shifted-ICCG の動作

`shift=1.0` は対角にシフトを加えない（$lpha = 1$）。
$arepsilon = 10^{-6}$ の微小正則化があるため、行列は正定値であり分解は成功する。

`auto_shift=True` を有効にすると、万一 $d_i$ が小さくなった場合に自動で $lpha$ を増加する。
`diagonal_scaling=True` は対角スケーリング $D^{-1/2} A D^{-1/2}$ で条件数を改善する。

In [3]:
# Shifted-ICCG (auto_shift + diagonal_scaling)
solver = SparseSolvSolver(
    a.mat, method="ICCG", freedofs=fes.FreeDofs(),
    tol=1e-8, maxiter=2000, shift=1.0,
    save_residual_history=True)
solver.auto_shift = True
solver.diagonal_scaling = True

t0 = time.time()
gfu_iccg = GridFunction(fes)
gfu_iccg.vec.data = solver * f.vec
t_iccg = time.time() - t0
res = solver.last_result

diff = curl(gfu_iccg) - B_ref
b_err = sqrt(Integrate(InnerProduct(diff, diff), mesh)) / B_norm

print(f"Shifted-ICCG: {res.iterations} 反復, 収束={res.converged}")
print(f"  計算時間: {t_iccg:.3f}s")
print(f"  B場相対誤差: {b_err:.4e}")

# 残差履歴の表示
hist = list(res.residual_history)
print()
print("残差履歴:")
for i in [0, 5, 10, 20, 30, min(40, len(hist)-1), len(hist)-1]:
    if i < len(hist):
        print(f"  iter {i:4d}: {hist[i]:.6e}")

Shifted-ICCG: 41 反復, 収束=True
  計算時間: 0.016s
  B場相対誤差: 3.3426e-09

残差履歴:
  iter    0: 1.000000e+00
  iter    5: 2.697735e-02
  iter   10: 3.237853e-04
  iter   20: 3.583339e-08
  iter   30: 4.632570e-05
  iter   40: 1.112939e-08
  iter   41: 7.936210e-09


## 検証2: BDDC vs ICCG

BDDC (Balancing Domain Decomposition by Constraints) は、
自由度をwirebasket/interfaceに分類し、ブロック消去で粗格子補正を構築する。

- 条件数: $\kappa = O((1 + \log(H/h))^2)$ → 反復数がメッシュ非依存
- curl-curl問題の近零固有値モードを粗格子で捕捉 → IC不要

In [4]:
# SparseSolv BDDC
t0_setup = time.time()
bddc = BDDCPreconditioner(a, fes, coarse_inverse="sparsecholesky")
t_setup = time.time() - t0_setup

t0_solve = time.time()
inv_bddc = CGSolver(mat=a.mat, pre=bddc, maxiter=500, tol=1e-8, printrates=False)
gfu_bddc = GridFunction(fes)
gfu_bddc.vec.data = inv_bddc * f.vec
t_solve = time.time() - t0_solve

diff_bddc = curl(gfu_bddc) - B_ref
b_err_bddc = sqrt(Integrate(InnerProduct(diff_bddc, diff_bddc), mesh)) / B_norm

print(f"BDDC: {inv_bddc.iterations} 反復")
print(f"  セットアップ: {t_setup:.3f}s, ソルブ: {t_solve:.3f}s")
print(f"  B場相対誤差: {b_err_bddc:.4e}")

# 比較表
print()
print("=" * 50)
print(f"{'ソルバー':>20} | {'反復':>6} | {'時間':>8} | {'B誤差':>12}")
print("-" * 50)
print(f"{'直接法':>20} | {'--':>6} | {t_direct:>7.3f}s | {'(参照)':>12}")
print(f"{'Shifted-ICCG':>20} | {res.iterations:>6} | {t_iccg:>7.3f}s | {b_err:>12.4e}")
print(f"{'SparseSolv BDDC':>20} | {inv_bddc.iterations:>6} | {t_setup+t_solve:>7.3f}s | {b_err_bddc:>12.4e}")
print("=" * 50)

BDDC: 23 反復
  セットアップ: 0.213s, ソルブ: 0.009s
  B場相対誤差: 5.8778e-09

                ソルバー |     反復 |       時間 |          B誤差
--------------------------------------------------
                 直接法 |     -- |   0.203s |         (参照)
        Shifted-ICCG |     41 |   0.016s |   3.3426e-09
     SparseSolv BDDC |     23 |   0.222s |   5.8778e-09


## 結論

1. **Shifted-ICCG** は `auto_shift=True` + `diagonal_scaling=True` で curl-curl 問題に対応可能
   - 微小正則化 $arepsilon = 10^{-6}$ があれば、shift=1.0 でも分解は成功する
2. **BDDC** は同じ問題を桁違いに少ない反復で解ける
   - 粗格子補正が低周波モードを直接捕捉するため
3. 次のノートブック (NB02) では、実問題 (コイル磁場解析) での性能差を示す